#### retreving all dataset.tt files and their commands 

In [62]:
import glob
import re
import pandas as pd
import numpy as np

path = '/users/mac/documents/programming/python/issai/thingpedia-common-devices-master/'
file_paths = [f for f in glob.glob(path + '**/dataset.tt', recursive=True)]

In [63]:
strs = []
for file_path in file_paths:
    f = open(file_path, "r")
    file_content = f.readlines()
    strs.append(file_content)

In [64]:
strs[0]

['dataset @com.phdcomics language "en" {\n',
 '    stream  := monitor (@com.phdcomics.get_post())\n',
 '    #_[utterances=["when there is a new post on phd comics",\n',
 '                   "when a new phd comics is posted",\n',
 '                   "when a new phd comic is published"]]\n',
 '    #[id=1691915]\n',
 '    #[name=""];\n',
 '\n',
 '    query  := @com.phdcomics.get_post()\n',
 '    #_[utterances=["posts on phd comics",\n',
 '                   "piled higher and deeper comics",\n',
 '                   "phdcomics posts",\n',
 '                   "posts on phdcomics",\n',
 '                   "phd comics"]]\n',
 '    #[id=1691916]\n',
 '    #[name=""];\n',
 '\n',
 '}']

In [65]:
for l in strs:
    for s in l:
        if 'Enum' in s:
            print("blah: ",s)
            s_temp = s.replace('Enum', 'BLAH')
            if 'Enum' in s_temp:
                print(s)
            

blah:      stream (p_state :Enum(open,closed))  := edge (monitor (@io.home-assistant.door.state())) on state == p_state

blah:      stream (p_name :String, p_state :Enum(open,closed))  := edge (monitor (@io.home-assistant.door(name=p_name).state())) on state == p_state

blah:      stream (p_state :Enum(detecting,not_detecting))  := edge (monitor (@io.home-assistant.motion.state())) on state == p_state

blah:      stream (p_name :String, p_state :Enum(detecting,not_detecting))  := edge (monitor (@io.home-assistant.motion(name=p_name).state())) on state == p_state

blah:      stream (p_state :Enum(plugged,unplugged))  := edge (monitor (@io.home-assistant.plug.state())) on state == p_state

blah:      stream (p_name :String, p_state :Enum(plugged,unplugged))  := edge (monitor (@io.home-assistant.plug(name=p_name).state())) on state == p_state

blah:      stream (p_power :Enum(on,off))  := edge (monitor (@light-bulb.power())) on power == p_power

blah:      action (p_power :Enum(on,off))  

In [184]:
def get_enum_values(contents, start_idx):
    l = contents[start_idx:].find("Enum(")
    r = contents[start_idx:].find(")")
    enum_values = []
    if (int(l) != -1 and int(r) != -1 and r > l): 
        enum_values = contents[start_idx:][l+len("Enum("):r]
        enum_values = enum_values.split(",")
    return enum_values

def find_var(command):
    ind = command.find('$')
    res = ''
    while (ind < len(command) and command[ind] != ' ' and command[ind] != '"'):
        res = res + command[ind]
        ind += 1
    return res

def subst_enum(utterances, enum_vals):
    result = []
    for command in utterances:
        if ('$' in command and len(enum_vals) > 0):
            var_name = find_var(command)
            for enum_val in enum_vals:
                command_rep = command.replace(var_name,enum_val)
                if ('$' not in command_rep): 
                    result.append(command_rep.strip().replace('"',''))
        elif ('$' not in command):
            result.append(command.strip().replace('"',''))
    return result

def get_filename(file_path):
    rev = ''
    i = int(len(file_path)-12)
    while (file_path[i] != '/'):
        rev += file_path[i]
        i -= 1
    res = ''.join(reversed(rev))
    return res

def check_str(char):
    if (char != ' ' and char != '"' and char != '(' and char != ')' and char != ',' and char != '{' and char != '}'):
        return True
    else:
        return False

def check_df(var_name, var_type, file_name):
    global var_df, df_ind
    if df_ind == 0: return True
    for ind in range(df_ind):
        if (var_df["var_name"].iloc[ind] == var_name and var_df["var_type"].iloc[ind] == var_type and var_df["file_name"].iloc[ind] == file_name):
            return False
    return True

def add_df(contents, i, j, st_ind, file_name):
    if (st_ind == j): return
    l = contents[i:][st_ind:].find("p_")
    if (contents[i:][l-1] != ' ' and contents[i:][l-1] != '('): return
    res_name = ''
    res_type = ''
    while (l < j and check_str(contents[i:][l])):
        res_name += contents[i:][l]
        l += 1
    l += 2
    while (l < j and check_str(contents[i:][l])):
        res_type += contents[i:][l]
        l += 1
    global df_ind, var_df
    if ("p_" in res_name and len(res_type) > 3 and check_df(res_name, res_type, file_name)):
        var_df.loc[df_ind, "var_name"] = res_name
        var_df.loc[df_ind, "var_type"] = res_type
        var_df.loc[df_ind, "file_name"] = file_name
        var_df.loc[df_ind, "var_ind"] = df_ind+1
        df_ind += 1
        add_df(contents, i, j, l, file_name)

def collect_commands(file_paths, command_type):
    commands_list = []
    for file_path in file_paths:
        f = open(file_path, "r")
        file_content = f.read()
        command_start_idx_lst = [i for i in range(len(file_content)) if file_content.startswith(command_type, i)]
        variable_set = []
        file_name = get_filename(file_path)
        for i in command_start_idx_lst:   
            j = file_content[i:].find("#_[utterances=[")
            k = file_content[i:].find("]]")
            utterances = file_content[i:][j+len("#_[utterances=["):k]
            utterances = utterances.split(',')
            add_df(file_content, i, j, 0, file_name)
            commands = []
            enum_vals = get_enum_values(file_content, i)

            if(len(enum_vals) > 0): 
                commands = subst_enum(utterances, enum_vals)
            else: 
                commands = [action_str.strip().replace('"','') for action_str in utterances if (action_str.strip().replace('"','') and ('$' not in action_str))]
            commands_list = commands_list + commands
    return commands_list   

In [185]:
df_ind = 0;
var_df = pd.DataFrame(columns = ["var_name","var_type","file_name", "var_ind"])

commands_action =  collect_commands(file_paths, 'action')
commands_query =  collect_commands(file_paths, 'query')

print("number of all action commands: ",len(commands_action))
print("number of all unique action commands: ",len(set(commands_action)))
print("here is the first action command: ", commands_action[0])
print("number of all query commands: ",len(commands_query))
print("number of all unique query commands: ",len(set(commands_query)))
print("here is the first query command: ", commands_query[0])
var_df = var_df.set_index('var_ind')
var_df.to_csv('commands_v2.csv')

number of all action commands:  350
number of all unique action commands:  331
here is the first action command:  tweet a picture
number of all query commands:  753
number of all unique query commands:  729
here is the first query command:  posts on phd comics


In [43]:
#check if there are numbers in any command
print([c for c in commands_action if (len(re.findall(r'\d+',c))>0)])
print([c for c in commands_query if (len(re.findall(r'\d+',c))>0)])

[]
['posts on matrix67', 'latest posts on matrix67', 'posts on matrix67 blog', 'cs294s class slides', 'cs294s slides', 'cs294s class materials', 'cs294w class slides', 'cs294w slides']


In [44]:
#removing short commands
commands_query_f = [l for l in commands_query if (len(l)>2)]
commands_query_f = [l for l in commands_query_f if (len(l.split())>1)]

In [45]:
new_commands_query_f = []
for c in commands_query_f:
    if "matrix67" in c:
        c = c.replace('matrix67','matrix sixty seven')
        print(c)
    if "cs294" in c:
        c = c.replace('cs294','cs two nine four ')
        print(c)
    new_commands_query_f.append(c)

posts on matrix sixty seven
latest posts on matrix sixty seven
posts on matrix sixty seven blog
cs two nine four s class slides
cs two nine four s slides
cs two nine four s class materials
cs two nine four w class slides
cs two nine four w slides


In [46]:
all_commands =  list(set(commands_action))+ list(set(new_commands_query_f))
print(len(all_commands))

1050


#### saving as .csv file

In [57]:
import pandas as pd
df = pd.DataFrame()
df["command_name"] = all_commands
df["command_id"] = range(1,len(all_commands)+1)
df["num_chars"] = [len(command) for command in all_commands]
df = df.set_index('command_id')
df.to_csv('csvs/commands_v1.csv')
df.head(10)

,command_name,num_chars
command_id,,
1,alert me by flashing the lights,31
2,open my window for me,21
3,increase volume on my speaker,29
4,order me a macchiato,20
5,put the curtains down,21
6,order me a americano,20
7,close my window,15
8,upload a picture to Imgur,25
9,open my curtains,16


#### saving as .mat file

In [48]:
import scipy.io as sio
sio.savemat('commands.mat', {"command_name": all_commands, 
                             "command_id" : range(len(all_commands)),
                             "num_chars": [len(command) for command in all_commands]}) 